# Interactive annotation with Pixano [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixano/pixano/blob/main/notebooks/annotation/interactive_annotation.ipynb)

This notebook allows you to pre-compute embeddings for a whole Pixano dataset using a deep learning model, and convert the model to ONNX.

You can then interactively annotate your dataset using that model in the Pixano Annotator.

## 1. Setting up

### Install dependencies

This notebook requires installing `pixano` as well the models from the complementary [`pixano-inference`](https://github.com/pixano/pixano-inference) module.

If you are running this notebook on your computer, we strongly recommend creating a virtual environment for using Pixano like so:

```shell
conda create -n pixano_env python=3.10
conda activate pixano_env
```

```shell
pip install pixano
pip install pixano-inference@git+https://github.com/pixano/pixano-inference
```

If you are running this notebook in Google Colab, run the cell below to install `pixano` and `pixano-inference`.

In [4]:
try:
  import google.colab
  ENV = "colab"
  !pip install pixano
  !pip install pixano-inference@git+https://github.com/pixano/pixano-inference
except:
  ENV = "jupyter"

### Load dependencies

In [1]:
from pathlib import Path

from pixano_inference import segment_anything

from pixano.apps import AnnotatorApp
from pixano.models import InferenceModel

### Download checkpoints
- Segment Anything Model:
    - `default` or `vit_h`: [ViT-H SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth).
    - `vit_l`: [ViT-L SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth).
    - `vit_b`: [ViT-B SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth).
    
## 2. Computing embeddings

### Select a model

- Loading a model:
    - Segment Anything Model
        - `model = segment_anything.SAM(checkpoint_path=Path("sam_vit_b_01ec64.pth"), size="b")`
        - `model = segment_anything.SAM(checkpoint_path=Path("sam_vit_h_4b8939.pth"), size="h")`
- Loading previously computed embeddings with model ID:
    - `model = segment_anything.SAM(checkpoint_path=Path("sam_vit_h_4b8939.pth"), size="h", id="230414_094523_SAM_ViT_H")`

By default, `device` is equal to `"cuda"` for PyTorch models and `"/GPU:0"` for TensorFlow models.

If you would like to run the model on your CPU, please add `device="cpu"` for PyTorch models or `device="/CPU:0"` for TensorFlow models in the arguments in the cell below.

In [2]:
model = segment_anything.SAM(checkpoint_path=Path("sam_vit_b_01ec64.pth"), size="b", device="cpu")

### Select a Pixano format dataset

If you haven't already, please refer to the [dataset notebooks](../dataset/) for information on how to import your dataset to Pixano format.

In [ ]:
library_dir = Path("datasets/")
dataset_dir = library_dir / "coco_instances"

views = ["image"]
splits = []

### Compute embeddings

In [ ]:
help(InferenceModel.process_dataset)

In [ ]:
embedding_dir = model.process_dataset(
    input_dir=dataset_dir,
    process_type="embed",
    views=views,
    splits=splits,
    batch_size=2,
)

## 3. Exporting model to ONNX

To use your model inside Pixano Annotator, you will need to convert it to ONNX.

In [ ]:
model.export_to_onnx(library_dir)

## 4. Annotating the dataset

With the generation complete, you can now browse your dataset with the Pixano Annotator and start annotating.

You can stop the Annotator app by restarting the notebook.

In [ ]:
annotator = AnnotatorApp(library_dir)

In [ ]:
annotator.display()